<a href="https://colab.research.google.com/github/mephi82/grad_data_analysis/blob/main/lstm_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import datasets, layers, utils, models, optimizers, preprocessing
import matplotlib.pyplot as plt
import numpy as np

In [2]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=top_words)
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,), (25000,))

In [3]:
np.array(list(map(len, X_train)))
set(y_train)

{0, 1}

In [4]:
# Retrieve the word index file mapping words to indices
word_index = datasets.imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
inverted_word_index = dict((i, word) for (word, i) in word_index.items())
# Decode the first sequence in the dataset
print(" ".join(inverted_word_index[i] for i in X_train[0]))
print(y_train[0])


1654784/1641221 [==============================] - 0s 0us/step
the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s and with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over and for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought and but of script you not while history he heart

In [5]:
# truncate and pad input sequences
max_review_length = 500
X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=max_review_length)

In [6]:
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((25000, 500), (25000,), (25000, 500), (25000,))

In [7]:
# create the model
embedding_vector_length = 32
def create_LSTM():
  model = models.Sequential()
  model.add(layers.Embedding(top_words, embedding_vector_length, input_length=max_review_length))
  model.add(layers.LSTM(128))

  # model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(layers.MaxPooling1D(pool_size=2))
  # model.add(layers.Dropout(0.5))
  # model.add(layers.Dense(256))
  # model.add(layers.Dropout(0.5))

  model.add(layers.Dense(1, activation='sigmoid'))
  return(model)


model = create_LSTM()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
hist = model.fit(X_train, y_train, validation_split = 0.1, epochs=3, batch_size=64)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 242,561
Trainable params: 242,561
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
352/352 [==============================] - 16s 28ms/step - loss: 0.5154 - accuracy: 0.7423 - val_loss: 0.3809 - val_accuracy: 0.8340
Epoch 2/3
352/352 [==============================] - 9s 27ms/step - loss: 0.3417 - accuracy: 0.8606 - val_loss: 0.3663 - val_accuracy: 0.8472
Epoch 3/3
352/352 [=====

In [10]:
def create_FCC():
  model = models.Sequential()
  model.add(layers.Embedding(top_words, embedding_vector_length, input_length=max_review_length))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))

  # model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(layers.MaxPooling1D(pool_size=2))
  # model.add(layers.Dropout(0.5))
  # model.add(layers.Dense(256))
  # model.add(layers.Dropout(0.5))

  model.add(layers.Dense(1, activation='sigmoid'))
  return(model)

model2 = create_FCC()
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
hist = model2.fit(X_train, y_train, validation_split = 0.1, epochs=3, batch_size=64)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 32)           160000    
                                                                 
 flatten (Flatten)           (None, 16000)             0         
                                                                 
 dense_5 (Dense)             (None, 128)               2048128   
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,208,257
Trainable params: 2,208,257
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
352/352 [==============================] - 2s 5ms/step - loss: 0.4203 - accuracy: 0.7861 - val_loss: 0.2985 - val_accuracy: 0.8740
Epoch 2/3
352/352 [===========

In [12]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.84%
Accuracy: 85.99%
